In [16]:
import folium
import leafmap.foliumap as leafmap
import geopandas as gpd
import pandas as pd

In [ ]:
#myshpfile = gpd.read_file('river_polygon.shx')
#myshpfile.to_file('river_polygon.geojson', driver='GeoJSON')

In [ ]:
m = leafmap.Map(location=[4,104], zoom_start=4)
m.add_geojson("river_polygon.geojson", layer_name="Malaysia Rivers")
m

In [21]:
file = "river_polygon.geojson"
river = gpd.read_file(file)

,geometry
0,"POLYGON ((100.18593 5.39230, 100.18633 5.39235..."
1,"POLYGON ((101.23443 2.90927, 101.23448 2.90974..."
2,"POLYGON ((103.67675 1.42505, 103.67731 1.42621..."
3,"POLYGON ((101.70081 2.95248, 101.70123 2.95271..."
4,"POLYGON ((101.68694 2.96379, 101.68694 2.96381..."
...,...
16539,"POLYGON ((101.11678 3.46457, 101.11680 3.46465..."
16540,"POLYGON ((101.11621 3.46538, 101.11647 3.46558..."
16541,"POLYGON ((103.91521 1.48638, 103.91533 1.48646..."
16542,"POLYGON ((103.91595 1.48578, 103.91598 1.48583..."


In [24]:
river = gpd.GeoDataFrame(river, geometry='geometry', crs = 'EPSG:4326').to_crs('EPSG:3857')
river.head()

,geometry
0,"POLYGON ((11152647.024 601156.030, 11152691.51..."
1,"POLYGON ((11269364.876 323998.054, 11269370.84..."
2,"POLYGON ((11541243.474 158652.365, 11541304.93..."
3,"POLYGON ((11321281.948 328814.369, 11321329.10..."
4,"POLYGON ((11319737.847 330074.571, 11319738.67..."


In [29]:
df = pd.read_csv('regression_data.csv')
df2 = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude), crs = 'EPSG:4326').to_crs('EPSG:3857')
df2.head()

,latitude,longitude,geometry
0,6.402858,100.098777,POINT (11142944.907 714251.095)
1,6.365790,100.188445,POINT (11152926.670 710098.960)
2,6.360280,100.212395,POINT (11155592.794 709481.714)
3,6.557335,100.314105,POINT (11166915.133 731557.973)
4,6.395771,100.230483,POINT (11157606.330 713457.232)


In [31]:
# Select a random location or random point
random_location = df2.iloc[360]

# Measure distance from release to each station
distances = river.geometry.distance(random_location.geometry)
distances

0        122182.246718
1        410397.735018
2        678740.837182
3        424459.348817
4        422003.039358
             ...      
16539    349757.269903
16540    349661.852656
16541    688343.335245
16542    688439.599416
16543    688851.943086
Length: 16544, dtype: float64

In [32]:
print("Mean distance: {}".format(distances.mean()))
print("Nearest flood point: {}".format(distances.min()))

Mean distance: 652383.9309346029
Nearest flood point: 4079.534366980356


In [33]:
# write a function to compute for the minimum distances from any flood point
random_location = df2.iloc[360]
distances = river.geometry.distance(random_location.geometry)
distances.min()

for i in range(0, len(df2)):
    distances = river.geometry.distance(df2.iloc[i].geometry)
    df2.loc[i, 'distance'] = distances.min()

In [34]:
df2.head()

,latitude,longitude,geometry,distance
0,6.402858,100.098777,POINT (11142944.907 714251.095),2866.669879
1,6.365790,100.188445,POINT (11152926.670 710098.960),2520.893227
2,6.360280,100.212395,POINT (11155592.794 709481.714),394.344896
3,6.557335,100.314105,POINT (11166915.133 731557.973),2065.659934
4,6.395771,100.230483,POINT (11157606.330 713457.232),2072.528557


In [38]:
# append the shortest distance to river column to the original dataframe
# df = pd.read_csv('regression_data.csv')
df['min_distance_river'] = df2['distance']
df.head()

,latitude,longitude,region,state,postcode,geometry,flood_risk,distance,kawasan_banjir,latitude_kb,longitude_kb,kedalaman_banjir,month_kb,altitude,slope,min_distance_river
0,6.402858,100.098777,Kangar,Perlis,1000,POINT (11142944.906933343 714251.0946146951),False,4245.737601,Kampung Nelayan,6.4040,100.1369,0.6,4.0,0,0.000000,2866.669879
1,6.365790,100.188445,Kangar,Perlis,1000,POINT (11152926.669637946 710098.9597010214),False,4532.533863,Kampung Sentua,6.4051,100.1981,0.2,10.0,4,2.075604,2520.893227
2,6.360280,100.212395,Kangar,Perlis,1000,POINT (11155592.793706343 709481.7137045604),False,5266.664356,Kampung Sentua,6.4051,100.1981,0.2,10.0,5,2.075599,394.344896
3,6.557335,100.314105,Kangar,Perlis,1000,POINT (11166915.132510776 731557.9727265495),False,8652.136771,Taman Beseri Jaya,6.5590,100.2364,0.6,5.0,59,0.933516,2065.659934
4,6.395771,100.230483,Kangar,Perlis,1000,POINT (11157606.329523865 713457.2321468537),False,3753.265280,Kampung Sentua,6.4051,100.1981,0.2,10.0,5,0.927410,2072.528557


In [39]:
df.tail()

,latitude,longitude,region,state,postcode,geometry,flood_risk,distance,kawasan_banjir,latitude_kb,longitude_kb,kedalaman_banjir,month_kb,altitude,slope,min_distance_river
19819,4.861237,115.406857,Lawas,Sarawak,98857,POINT (12847032.566424089 541800.8265352033),True,317.342199,Bandar Lawas,4.859,115.4051,0.15,9.0,11,4.143626,315.745534
19820,4.861249,115.406919,Lawas,Sarawak,98857,POINT (12847039.490496416 541802.1935578815),True,322.719373,Bandar Lawas,4.859,115.4051,0.15,9.0,10,2.931484,312.397021
19821,4.861206,115.406885,Lawas,Sarawak,98857,POINT (12847035.67223788 541797.4409309261),True,316.622942,Bandar Lawas,4.859,115.4051,0.15,9.0,11,4.143626,311.157523
19822,4.861273,115.406945,Lawas,Sarawak,98857,POINT (12847042.395935126 541804.9198945377),True,326.665809,Bandar Lawas,4.859,115.4051,0.15,9.0,10,2.931484,312.653287
19823,4.861271,115.406852,Lawas,Sarawak,98857,POINT (12847032.043222483 541804.6331057694),True,320.029117,Bandar Lawas,4.859,115.4051,0.15,9.0,11,4.143626,319.015807


In [40]:
df.to_csv('regression_data.csv', index=False)